In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": ".......................",
  "fs.azure.account.oauth2.client.secret": "...................................",
  "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/be50fa3b-c833-4bb4-b7ee-9a5b37bcfc2b/oauth2/token"}


dbutils.fs.mount(
    source = "abfss://tokyo-olympic-data@storagetokyoolympic117.dfs.core.windows.net", #container@storegeaccountname.dfs.core.windows.net
    mount_point = "/mnt/storagetokyoolympic117",
    extra_configs = configs)
    

In [0]:
%fs
ls "/mnt/storagetokyoolympic117"

## IN DATABRICKS, SPARK IS ALREADY INTEGRATED AND WE DO NOT SET THE SPARK SESSION !!!

In [0]:
spark #TO SEE THE DETAILS OF THE SPARK SESSION

In [0]:
athletes = spark.read.csv("dbfs:/mnt/storagetokyoolympic117/raw-data//athletes.csv", header=True)
coaches = spark.read.csv("dbfs:/mnt/storagetokyoolympic117/raw-data//coaches.csv", header=True)
entriesgender = spark.read.csv("dbfs:/mnt/storagetokyoolympic117/raw-data//entriesgender.csv", header=True)
medals = spark.read.csv("dbfs:/mnt/storagetokyoolympic117/raw-data//medals.csv", header=True)
teams = spark.read.csv("dbfs:/mnt/storagetokyoolympic117/raw-data//teams.csv", header=True)

In [0]:
athletes.show()

In [0]:
athletes.printSchema()
coaches.printSchema()
entriesgender.printSchema()  # age as string is not good, we will change it
medals.printSchema() # gold, silver, bronze as string is not good, we will change it
teams.printSchema()

In [0]:
entriesgender = entriesgender.withColumn("Female", entriesgender["Female"].cast("int"))
entriesgender = entriesgender.withColumn("Male", entriesgender["Male"].cast("int"))

#### Now we should change the data type of the columns bronze, silver and gold for medals. instead of doind this, we set "inferSchema" in the spark.read command to ask spark to assign an appropriate datatype to each column based on its content !!!

In [0]:
medals = spark.read.csv("dbfs:/mnt/storagetokyoolympic117/raw-data//medals.csv", header=True)  # this syntax is not valid anymore
medals = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("dbfs:/mnt/storagetokyoolympic117/raw-data//medals.csv")  #this is the new syntax

teams = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("dbfs:/mnt/storagetokyoolympic117/raw-data//teams.csv")  #this is the new syntax

In [0]:
# find the top countries with the highest number of medals
top_gold_medals_countries = medals.orderBy("Gold", ascending= False).select("TeamCountry", "Gold")

top_gold_medals_countries.show()

In [0]:
entriesgender.show()

In [0]:

top_gold_medals_countries = medals.orderBy("Gold", ascending= False).select("TeamCountry", "Gold")
top_entries_bygender = entriesgender.orderBy("Female", "Male", ascending = False).select("Discipline", "Female", "Male")
top_entries_bygender.show()

In [0]:
# calculate the weights of entries by gender for each discipline
weights_by_gender = entriesgender.withColumn(
    "%_entries_female", entriesgender["Female"] / entriesgender["Total"]
    ).withColumn(
        "%_entries_male", entriesgender["Male"] / entriesgender["Total"]
    )
weights_by_gender.show()

In [0]:
athletes.write.option("header", "true").csv("dbfs:/mnt/storagetokyoolympic117/transformed-data/athletes")

athletes.write.mode("overwrite").option("header", "true").csv("dbfs:/mnt/storagetokyoolympic117/transformed-data/athletes") # to overwrite if you run the code again

athletes.write.repartition(3).mode("overwrite").option("header", "true").csv("dbfs:/mnt/storagetokyoolympic117/transformed-data/athletes") # if you want to split the output into 3 files because it is too big

In [0]:
coaches.write.mode("overwrite").option("header", "true").csv("dbfs:/mnt/storagetokyoolympic117/transformed-data/coaches")
entriesgender.write.mode("overwrite").option("header", "true").csv("dbfs:/mnt/storagetokyoolympic117/transformed-data/entriesgender")
medals.write.mode("overwrite").option("header", "true").csv("dbfs:/mnt/storagetokyoolympic117/transformed-data/medals")
teams.write.mode("overwrite").option("header", "true").csv("dbfs:/mnt/storagetokyoolympic117/transformed-data/teams")


